<a href="https://colab.research.google.com/github/syauqyideas/Colab_Runner_FluidX3D/blob/main/FluidX3D_Colab_Plane_External_Flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>Copy FluidX3d Directory and make Project File

In [ ]:
!pip install IPython
!pip install k3d
!pip install numpy-stl numpy matplotlib
!jupyter nbextension install --py --user k3d
!jupyter nbextension enable --py --user k3d

import IPython #at some point, i just bored to see the dirty output
from IPython.display import clear_output
import k3d
from google.colab import output

output.enable_custom_widget_manager()

k3d.switch_to_text_protocol()

!mkdir FluidX3D
%cd FluidX3D
!sudo apt install build-essential
!pip install numpy-stl
!wget https://github.com/ProjectPhysX/FluidX3D/archive/refs/heads/master.zip
!unzip master.zip
clear_output()
%cd FluidX3D-master

<h2>Install OpenCL Driver and reinstall nvidia-driver

In [ ]:
# doing this line in internal xterm to prevent error
#!pip install colab-xterm
#%load_ext colabxterm
#%xterm

#insert this code :
!pip install numpy-stl
!sudo apt update && sudo apt upgrade -y
!sudo apt install -y g++ git make ocl-icd-libopencl1 ocl-icd-opencl-dev #nvidia-driver-550
clear_output() #you can take away this

In [ ]:
!apt install nvidia-driver-555
!sudo apt update && sudo apt upgrade


clear_output() #you can take away this

!nvcc --version #it is just check cuda versi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
!nvcc --version

<h1><strong>Case Setup</h1>

<h2><strong>STL Mesh Manipulation

<h3>Read STL<h3>

<h3>Case Type</h3>

In [ ]:
Type = 'External Flow' #@param ['External Flow','Benchmark','Coming Soon']{allow-input : false}

<h3>Case Boundary Condition

In [ ]:
speed_timespeed = 1 #@param
time_timespeed = 1 #@param
res_x = 1 #@param
res_y = 1 #@param
res_z = 1 #@param
maximum_sim_time = 1 #@param
nu_value = 1 #@param
rho     = 1 #@param
res_multiplicaton = 2000 #@param
nu      = 1 #@param
rho     = 1 #@param
reynold_number = 1 #@param
domain_length = 1 #@param
domain_width = 1 #@param

stl_file_source = '/content/concord_cut_large.stl' #@param {allow-input : true}


<h3>STL File Verication

In [ ]:
# @title
import stl
from stl import mesh
import numpy as np

def get_stl_bounds(stl_file_source):
    """
    Membaca file STL dan mengembalikan nilai min/max koordinat untuk setiap sumbu.

    Args:
        file_path (str): Path lengkap ke file STL.

    Returns:
        dict: Kamus yang berisi nilai min dan max untuk X, Y, Z.
              Contoh: {'x_min': ..., 'x_max': ..., 'y_min': ..., 'y_max': ..., 'z_min': ..., 'z_max': ...}
              Mengembalikan None jika file tidak ditemukan atau ada kesalahan.
    """
    try:
        # Memuat mesh dari file STL
        your_mesh = mesh.Mesh.from_file(stl_file_source)

        # Koordinat titik-titik (vertices) dari mesh
        # your_mesh.vectors berisi array dari segitiga. Setiap segitiga memiliki 3 titik.
        # Setiap titik memiliki 3 koordinat (x, y, z).
        # Bentuk data: (jumlah_segitiga, 3_titik_per_segitiga, 3_koordinat_per_titik)
        all_points = your_mesh.vectors.reshape(-1, 3)

        # Mendapatkan nilai min dan max untuk setiap sumbu
        x_min = np.min(all_points[:, 0])
        x_max = np.max(all_points[:, 0])

        y_min = np.min(all_points[:, 1])
        y_max = np.max(all_points[:, 1])

        z_min = np.min(all_points[:, 2])
        z_max = np.max(all_points[:, 2])

        return {
            'x_min': x_min, 'x_max': x_max,
            'y_min': y_min, 'y_max': y_max,
            'z_min': z_min, 'z_max': z_max
        }

    except FileNotFoundError:
        print(f"Error: File tidak ditemukan di '{file_path}'")
        return None
    except Exception as e:
        print(f"Terjadi kesalahan saat membaca file STL: {e}")
        return None

if __name__ == "__main__":

    bounds = get_stl_bounds(stl_file_source)

    if bounds:
        print("\n--- Batas Koordinat dari File STL ---")
        print(f"X-Axis: Min = {bounds['x_min']:.4f}, Max = {bounds['x_max']:.4f}")
        print(f"Y-Axis: Min = {bounds['y_min']:.4f}, Max = {bounds['y_max']:.4f}")
        print(f"Z-Axis: Min = {bounds['z_min']:.4f}, Max = {bounds['z_max']:.4f}")
    else:
        print("\nTidak dapat mendapatkan batas koordinat. Silakan periksa path file dan integritas file STL.")

    delta_x = bounds['x_max'] - bounds['x_min']
    delta_y = bounds['y_max'] - bounds['y_min']
    delta_z = bounds['z_max'] - bounds['z_min']
    print('Besar file STL')
    print(f'Delta_X = {delta_x}')
    print(f'Delta_Y = {delta_y}')
    print(f'Delta_Z = {delta_z}')

    if delta_x > domain_length:
      print(f'Panjang domain terlalu kecil dibandingkan file STL')
    if delta_y > domain_height:
      print(f'Tinggi domain terlalu kecil dibandingkan file STL')
    if delta_z > domain_width:
      print(f'Lebar domain terlalu besar dibandingkan file STL')

<h3>Code Declaration and Export of setup.cpp and defines.hpp

In [ ]:
# @title
def create_cpp_file(filename, content):
    """
    Membuat atau menimpa file .cpp dengan nama dan konten yang diberikan.

    Args:
        filename (str): Nama file .cpp yang akan dibuat (misal: "program_saya.cpp").
        content (str): String yang berisi kode C++ yang akan ditulis ke file.
    """
    if not filename.endswith(".cpp"):
        filename += ".cpp"
    try:
        with open(filename, 'w') as f:
            f.write(content)
        print(f"File '{filename}' berhasil dibuat atau diperbarui.")
    except IOError as e:
        print(f"Terjadi kesalahan saat menulis file '{filename}': {e}")

if __name__ == "__main__":
    print("--- Aplikasi Pembuat File C++ dengan Variabel Python ---")
    # Perhatikan 'f' sebelum tanda kutip tiga untuk membuat f-string
    cpp_code_to_write = f"""
#include <iostream>
#include <string>
#include <vector>

void main_setup() {{ // External FLow No Ground Basis Code
	//  define simulation box size, viscosity and volume force //
	const uint3 lbm_N = resolution(float3({res_x}f, {res_y}f, {res_z}f), 2084u); // input: simulation box aspect ratio and VRAM occupation in MB, output: grid resolution
	const float si_u = {speed_timespeed}f/{time_timespeed}f;
	const float si_length={domain_length}f, si_width={domain_width}f;
	const float si_T = {maximum_sim_time}f;
	const float si_nu=1.48E-5f, si_rho=1.225f;
	const float lbm_length = (float)lbm_N.y;
	const float lbm_u = 0.075f;
	units.set_m_kg_s(lbm_length, lbm_u, 1.0f, si_length, si_u, si_rho);
	const float lbm_nu = units.nu(si_nu);
	const ulong lbm_T = units.t(si_T);
	print_info("Re = "+to_string(to_uint(units.si_Re(si_width, si_u, si_nu))));
	LBM lbm(lbm_N, 1u, 1u, 1u, lbm_nu);
	// ###################################################################################### define geometry ######################################################################################
	const float3 center = float3(lbm.center().x, lbm.center().y, lbm.center());
	const float3x3 rotation = float3x3(float3(1, 0, 0), radians(-10.0f))*float3x3(float3(0, 0, 1), radians(90.0f))*float3x3(float3(1, 0, 0), radians(90.0f));
	lbm.voxelize_stl(get_exe_path()+"../stl/concord_cut_large.stl", center, rotation, lbm_length); // https://www.thingiverse.com/thing:1176931/files
	const uint Nx=lbm.get_Nx(), Ny=lbm.get_Ny(), Nz=lbm.get_Nz(); parallel_for(lbm.get_N(), [&](ulong n) {{ uint x=0u, y=0u, z=0u; lbm.coordinates(n, x, y, z);
		if(lbm.flags[n]!=TYPE_S) lbm.u.y[n] = lbm_u;
		if(x==0u||x==Nx-1u||y==0u||y==Ny-1u||z==0u||z==Nz-1u) lbm.flags[n] = TYPE_E; // all non periodic
	}}); // ####################################################################### run simulation, export images and data ##########################################################################
	lbm.graphics.visualization_modes = VIS_FLAG_SURFACE|VIS_Q_CRITERION;
	lbm.run(0u, lbm_T); // initialize simulation
	lbm.write_status();
	while(lbm.get_t()<=lbm_T) }} // main simulation loop
"""

    # --- AKHIR DARI DEKLARASI KODE C++ ---

    # Tentukan nama file C++ yang akan dibuat
    output_filename = "setup.cpp"

    # Panggil fungsi untuk membuat file
    create_cpp_file(output_filename, cpp_code_to_write)

    print(f"\nFile C++ '{output_filename}' telah berhasil dibuat dengan nilai variabel Python.")
    print("Anda sekarang bisa mengkompilasi dan menjalankannya menggunakan perintah seperti:")
    print(f"  !g++ {output_filename} -o dynamic_cpp_program")
    print("  !./dynamic_cpp_program")

<h2>Edit your Project first depending on your Need</h2>
<h3>Read documentation First on /content/fluidx1/FluidX3D-master/DOCUMENTATION.md

<h2>Run Project and Render

In [ ]:
%cd /content/fluidx1/FluidX3D-master
!./make.sh

<h2>Convert Image Sequence to mp4

In [ ]:
%cd /content
%cd /content/FluidX3D/FluidX3D-master/bin/export
#!ls -v image-*.png | awk '{print "file \047" $0 "\047"}' > imagelist.txt
#!head -n 20 imagelist.txt > imagelist_test.txt
#!ffmpeg -f concat -i imagelist_test.txt -framerate 25 -c:v libx264 -pix_fmt yuv420p output_test.mp4
!ffmpeg -framerate 24 -pattern_type glob -i "/content/FluidX3D/FluidX3D-master/bin/export/back/image-*.png" -c:v h264_nvenc -qp 18 -pix_fmt yuv420p output_gpu_q_back_.mp4
!ffmpeg -framerate 24 -pattern_type glob -i "/content/FluidX3D/FluidX3D-master/bin/export/follow/image-*.png" -c:v h264_nvenc -qp 18 -pix_fmt yuv420p output_gpu_q_follow.mp4
!ffmpeg -framerate 24 -pattern_type glob -i "/content/FluidX3D/FluidX3D-master/bin/export/front/image-*.png" -c:v h264_nvenc -qp 18 -pix_fmt yuv420p output_gpu_q_front.mp4
!ffmpeg -framerate 24 -pattern_type glob -i "/content/FluidX3D/FluidX3D-master/bin/export/side/image-*.png" -c:v h264_nvenc -qp 18 -pix_fmt yuv420p output_gpu_q_side.mp4
!ffmpeg -framerate 24 -pattern_type glob -i "/content/FluidX3D/FluidX3D-master/bin/export/top/image-*.png" -c:v h264_nvenc -qp 18 -pix_fmt yuv420p output_gpu_q_top.mp4
!ffmpeg -framerate 24 -pattern_type glob -i "/content/FluidX3D/FluidX3D-master/bin/export/wing/image-*.png" -c:v h264_nvenc -qp 18 -pix_fmt yuv420p output_gpu_q_wing.mp4
#!ffmpeg -f concat -safe 0 -i imagelist.txt -framerate 25 -c:v h264_nvenc -pix_fmt yuv420p output_gpu.mp4

<h2>Copy result file to your Drive

In [ ]:
from IPython.display import display, Javascript

# Function to create an alert
def create_alert(message):
    display(Javascript(f'alert("{message}")'))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create an alert
create_alert("Start Copying")

!mkdir /content/drive/MyDrive/fluidx1output
!cp -r /content/fluidx1/FluidX3D-master/bin/export /content/drive/MyDrive/fluidx1output

In [ ]:
!cp /content/FluidX3D/FluidX3D-master/bin/export/output_gpu_q* /content/drive/MyDrive/OpenFoam/

<h1>For Interactive Graphic

In [28]:
#STL Visualizer -  fit your STL right on google Colab

from stl import mesh
import numpy as np
import k3d

# 1. Muat file STL
file_path = '/content/F 22 RAPTOR_Coarse.stl' # Ganti dengan nama file Anda
file_path_stl_template = '/content/f15RotatedConcorde.stl'

your_mesh = mesh.Mesh.from_file(file_path)
mesh_template = mesh.Mesh.from_file(file_path_stl_template)

# 2. Ekstrak Vertices dan Indices (Faces) dalam format yang dibutuhkan K3D

# Vertices: Meratakan array (N_faces, 3, 3) menjadi (N_total_pts, 3)
vertices = your_mesh.vectors.reshape(-1, 3).astype(np.float32)
vertices_template = mesh_template.vectors.reshape(-1, 3).astype(np.float32)

# Indices: Membuat indeks (faces) dari vertices yang baru diratakan
# Setiap 3 baris dari array 'vertices' baru merupakan 1 segitiga
indices = np.arange(len(vertices)).reshape(-1, 3).astype(np.uint32)
indices_template = np.arange(len(vertices_template)).reshape(-1, 3).astype(np.uint32)

# 3. Buat Plot K3D
plot = k3d.plot(grid_visible=True, camera_auto_fit=True)
plot.display()

# 4. Plot template mesh (Menggunakan data STL yang diekstrak)
# Jika Anda ingin menampilkan mesh STL
plot += k3d.mesh(
    vertices,
    indices,
    opacity=0.75,
    wireframe=False,
    color=0xAAAAAA # Contoh warna abu-abu
)

#5. Plot your mesh
plot += k3d.mesh(
    vertices_template,
    indices_template,
    opacity=0.75,
    wireframe=False,
    color=0xffeb3b # Yellow is template STL - fit your mesh into STL
)



# ----------------------------------------------------------------------
# CATATAN PENTING UNTUK k3d.line:
# File STL tidak memiliki 'streamlines' (garis) atau atribut skalar 'v'.
# Baris kode di bawah ini akan menyebabkan error kecuali Anda secara manual
# mendefinisikan variabel 'lines' dan 'v' dari sumber lain.
# ----------------------------------------------------------------------

# plot += k3d.line(data_lines, ...)  # <-- Harus menggunakan variabel array NumPy yang berisi data garis
# plot += k3d.mesh(..., attribute=data_v, ...) # <-- Harus menggunakan variabel array NumPy yang berisi atribut skalar

#print(f"Mesh dari '{file_path}' telah diplot.")

Output()